### MNISTでLeNetを学習する

このプログラムでは，
- [torchvision.datasets](https://pytorch.org/docs/stable/torchvision/datasets.html)からのデータセット読み込み
- モデルの定義
- ロス関数・最適化関数の宣言
- 学習・テスト

を行った．

In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import transforms, datasets

### 定数定義
バッチサイズは100，
学習率は0.01，
モーメンタムは0.9とした．

In [2]:
root = 'data'

batch_size = 100
lr = .01
momentum = .9

log_interval = 50

### データ読み込み

transformは，データ読み込み時に画像をTensorに変換し，正規化する．  
学習ではシャッフルし，テストではシャッフルしない．

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

train_set = datasets.MNIST(root=root, train=True, transform=transform, download=True)
test_set = datasets.MNIST(root=root, train=False, transform=transform, download=True)

train_loader = DataLoader(train_set, num_workers=2, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, num_workers=2, batch_size=batch_size, shuffle=False)

### モデル定義

MNIST用のモデルである[LeNet](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf)を使う．

In [4]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(in_features=16*4*4, out_features=120)
        self.fc2 = nn.Linear(120, 64)
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        # shape of x is (batch_size, 1, 28, 28)
        x = F.relu(self.conv1(x))
        # (batch_size, 6, 24, 24)
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        # (batch_size, 6, 12, 12)
        x = F.relu(self.conv2(x))
        # (batch_size, 16, 8, 8)
        x = F.max_pool2d(x, 2, 2)
        # (batch_size, 16, 4, 4)
        x = x.view(x.shape[0], -1)
        # (batch_size, 16*4*4)
        x = F.relu(self.fc1(x))
        # (batch_size, 120)
        x = F.relu(self.fc2(x))
        # (batch_size, 64)
        x = self.fc3(x)
        # (batch_size, 10)
        return x
    
model = LeNet()
model.cuda()  # use GPU

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

### Loss, Optimizer
ロスは[Cross Entropy](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss)，
最適化は[Momentum SGD](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD)を使う．

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

### 学習関数

In [6]:
def train(epoch):
    model.train()
    for batch_idx, (data, label) in enumerate(train_loader):
        data, label = data.cuda(), label.cuda()
        optimizer.zero_grad()
        
        out = model(data)
        
        loss = criterion(out, label)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{:5}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))  

### テスト関数

In [7]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, label in test_loader:
            data, label = data.cuda(), label.cuda()
            out = model(data)
            test_loss += F.nll_loss(out, label, size_average=False).item()
            pred = out.max(1, keepdim=True)[1]
            correct += pred.eq(label.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

### 学習
20エポック学習  
各エポックでテスト

In [8]:
epochs = 20

for i in range(epochs):
    train(i + 1)
    test()

Train Epoch: 1 [    0/60000 (0%)]	Loss: 2.308331
Train Epoch: 1 [ 5000/60000 (8%)]	Loss: 2.276386
Train Epoch: 1 [10000/60000 (17%)]	Loss: 1.598979
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.371113
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.188731
Train Epoch: 1 [25000/60000 (42%)]	Loss: 0.309316
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.040514
Train Epoch: 1 [35000/60000 (58%)]	Loss: 0.136319
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.115782
Train Epoch: 1 [45000/60000 (75%)]	Loss: 0.103415
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.059844
Train Epoch: 1 [55000/60000 (92%)]	Loss: 0.280316

Test set: Average loss: -11.4151, Accuracy: 9735/10000 (97%)

Train Epoch: 2 [    0/60000 (0%)]	Loss: 0.195554
Train Epoch: 2 [ 5000/60000 (8%)]	Loss: 0.143854
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.095130
Train Epoch: 2 [15000/60000 (25%)]	Loss: 0.172424
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.143943
Train Epoch: 2 [25000/60000 (42%)]	Loss: 0.087300
Train Epoch: 2 [30000/60000 (50%)]	Loss: 